# Modelo de Classificação (Youtube Video Dataset)
https://www.kaggle.com/datasets/rahulanand0070/youtubevideodataset

In [368]:
import pandas as pd
import numpy as np
import re

# NLTK
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# SKLearn
from sklearn.model_selection import train_test_split

## Preparação do Dataset

A partir do .csv informado, vamos preparar o dataset para os algoritmos. Nesse caso, vamos remover valores nulos e filtrar as colunas de interesse.

In [369]:
df = pd.read_csv('datasets/youtube_video_dataset.csv')
df.head()

Title              Videourl  \
0  Madagascar Street Food!!! Super RARE Malagasy ...  /watch?v=EwBA1fOQ96c   
1            42 Foods You Need To Eat Before You Die  /watch?v=0SPwwpruGIA   
2                Gordon Ramsay’s Top 5 Indian Dishes  /watch?v=upfu5nQB2ks   
3        How To Use Chopsticks - In About A Minute 🍜  /watch?v=xFRzzSF_6gk   
4                       Trying Indian Food 1st Time!  /watch?v=K79bXtaRwcM   

  Category                                        Description  
0     Food  🎥GIANT ALIEN SNAIL IN JAPAN! » https://youtu.b...  
1     Food  This is the ultimate must-try food bucket list...  
2     Food  We found 5 of the best and most interesting In...  
3     Food  You're most likely sitting in a restaurant wit...  
4     Food  HELP SUPPORT SINSTV!! Shop Our Sponsors!\nLast...

In [370]:
colunas = list()
for coluna in df.columns:
    colunas.append(coluna)
print("Colunas:", " ".join(colunas))
print("Número de Linhas:", df.shape[0])
# Removendo as colunas que não são interessantes (nesse caso, apenas importa "Title" e "Category")
df = df.iloc[:, [0, 2, 3]]
colunas = list()
for coluna in df.columns:
    colunas.append(coluna)
print("Colunas:", " ".join(colunas))
print("Número de Linhas com Valores Nulos:", df.isna().sum().sum())
# Retirando linhas com valores nulos 
df = df.dropna()
print("Número de Linhas Após Remoção de Nulos:", df.shape[0])
print("Número de Linhas com Valores Nulos (Verificação):", df.isna().sum().sum())

Colunas: Title Videourl Category Description
Número de Linhas: 11211
Colunas: Title Category Description
Número de Linhas com Valores Nulos: 83
Número de Linhas Após Remoção de Nulos: 11128
Número de Linhas com Valores Nulos (Verificação): 0


### Novo dataset com as colunas novas e valores removidos:

In [371]:
df

Title   Category  \
0      Madagascar Street Food!!! Super RARE Malagasy ...       Food   
1                42 Foods You Need To Eat Before You Die       Food   
2                    Gordon Ramsay’s Top 5 Indian Dishes       Food   
3            How To Use Chopsticks - In About A Minute 🍜       Food   
4                           Trying Indian Food 1st Time!       Food   
...                                                  ...        ...   
11206  art journal | shimmer sprays, stencils, collag...  Art&Music   
11207                   Ar-Tea Collage * Mixed Media Art  Art&Music   
11208  DIY Mixed Media Art Collage Greeting Cards / M...  Art&Music   
11209  Art Collage Process DecoJournal using Rice Pap...  Art&Music   
11210  Journal Collage Process using Magazine Cut Out...  Art&Music   

                                             Description  
0      🎥GIANT ALIEN SNAIL IN JAPAN! » https://youtu.b...  
1      This is the ultimate must-try food bucket list...  
2      We found 5 of the best and most interesting In...  
3      You're most likely sitting in a restaurant wit...  
4      HELP SUPPORT SINSTV!! Shop Our Sponsors!\nLast...  
...                                                  ...  
11206  Step by step video on creating an art journal ...  
11207  By: Ilene McInnes,\nMixed media Art and inspir...  
11208  Make your own Mixed Media Greeting Cards\n\nHe...  
11209  Art Collage Process DecoJournal using Rice Pap...  
11210  Using magazine, scrapbook paper, recycled old ...  

[11128 rows x 3 columns]

Abaixo, foi verificado quantas categorias existem e qual a frequência de cada categoria. Como é possível observar, a mais comum é a de Viagem, enquanto a menos comum é a de História. Nesse caso, vamos mapear as classes aqui informadas para uma representação numérica.

In [372]:
df['Category'].value_counts()

travel blog           2200
Science&Technology    2074
Food                  1828
manufacturing         1699
Art&Music             1682
History               1645
Name: Category, dtype: int64

In [373]:
lista_categorias = df['Category'].value_counts().index.to_list()
dict_mapeamento = dict()
for i in range(len(lista_categorias)):
    dict_mapeamento[lista_categorias[i]] = i
df['Category'] = df['Category'].map(dict_mapeamento)
df.head()

Title  Category  \
0  Madagascar Street Food!!! Super RARE Malagasy ...         2   
1            42 Foods You Need To Eat Before You Die         2   
2                Gordon Ramsay’s Top 5 Indian Dishes         2   
3        How To Use Chopsticks - In About A Minute 🍜         2   
4                       Trying Indian Food 1st Time!         2   

                                         Description  
0  🎥GIANT ALIEN SNAIL IN JAPAN! » https://youtu.b...  
1  This is the ultimate must-try food bucket list...  
2  We found 5 of the best and most interesting In...  
3  You're most likely sitting in a restaurant wit...  
4  HELP SUPPORT SINSTV!! Shop Our Sponsors!\nLast...

## Bag of Words

O bag of words é uma representação de um texto, na forma de conjunto de palavras. Essa representação é bastante utilizada porque ela transforma o texto em um conjunto de informações mensuráveis (feature extraction). Nesse caso, foi utilizado um dicionário cujas chaves representam as palavras do vocabulário (qualquer palavra presente em algum título do dataset que não é stopword e possui mais de 3 caracteres) e cujos valores são 1 se essa palavra está presente no título e 0 caso contrário. 

### Criação do Vocabulário

In [374]:
#import nltk
#nltk.download('punkt')
#https://datascience.stackexchange.com/questions/25004/text-classifier-with-multiple-bag-of-words
from langdetect import detect
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer() # acurácia 93% com Tfid, 92% com CountVectorizer

lista_stopwords = set(stopwords.words('english'))
ps = PorterStemmer()

vocabulario = list()

freq_idiomas = dict()

df['Title'] = df['Title'].apply(lambda x: x.lower()) # Tirando letras minúsculas
df['Title'] = df['Title'].apply(lambda x: re.sub("[^a-zA-Z]"," ", x)) # Filtrando símbolos

lista_frases = list()

for i, row in df.iterrows():
    lista_palavra = word_tokenize(row['Title'])
    lista_palavra.extend(word_tokenize(row['Description']))
    lista_novo_titulo = []
    for palavra in lista_palavra:
        if palavra not in lista_stopwords:
            palavra = ps.stem(palavra) # Stemming da palavra
            vocabulario.append(palavra)
            lista_novo_titulo.append(palavra)
    string = " ".join(lista_novo_titulo)
    lista_frases.append(string)
    df.at[i, 'Title'] = lista_novo_titulo

array_frases = np.array(lista_frases)
bag = vectorizer.fit_transform(array_frases)

In [375]:
# # Por enquanto o BOW desse jeito tá muito pesado, não está valendo

# array_vazia = ([0] * len(vocabulario))
# array_bow = {}
# for palavra in vocabulario:
#     array_bow[palavra] = 0

# lista_bows = list()
# for i, row in df.iterrows():
#     lista_titulo = row['Title']
#     array_bow_copia = array_bow.copy()
#     for palavra in lista_titulo:
#         array_bow_copia[palavra] = 1
#     lista_bows.append(array_bow_copia)
    
# df['BOW'] = lista_bows
# df_com_titulo = df # Guardando esse dataframe com as três colunas
# df = df.iloc[:, [1, 2]] # Filtrando apenas a Categoria e o BOW
# df

## Separação do Dataset entre Treinamento e Teste

In [376]:
categorias = df['Category'].values
X_train, X_test, y_train, y_test = train_test_split(bag, categorias, stratify=categorias, test_size=0.33)

## Regressão Logística

In [377]:
# # https://vitalflux.com/text-classification-bag-of-words-model-python-sklearn/
# import warnings
# warnings.filterwarnings('ignore')

# from sklearn.linear_model import LogisticRegression
# from sklearn import metrics
# from sklearn.metrics import classification_report

# lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr')
# lr.fit(x_train, y_train)
# y_test_predict = lr.predict(x_test)
# y_train_predict = lr.predict(x_train)
# # print("Acurácia da Regressão Logística: %.3f" %metrics.accuracy_score(y_test, y_predict))
# # https://towardsdatascience.com/how-to-check-if-a-classification-model-is-overfitted-using-scikit-learn-148b6b19af8b
# # http://computacaointeligente.com.br/outros/intro-sklearn-part-3/

### Cross Validation

In [378]:
# # https://medium.com/@edubrazrabello/cross-validation-avaliando-seu-modelo-de-machine-learning-1fb70df15b78
# from sklearn.model_selection import cross_val_score

# scores = cross_val_score(lr, bag, categorias, cv=10, scoring='accuracy')
# print("Acurácia da Regressão Logística: %0.2f%%" % (scores.mean()*100)) ## talvez botar desvio padrao .std *2?

### Matriz de Confusão

In [379]:
# # https://www.jcchouinard.com/confusion-matrix-in-scikit-learn/
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, plot_confusion_matrix

# cm = confusion_matrix(y_test, y_test_predict)
# plot_confusion_matrix(lr, x_test, y_test, cmap=plt.cm.Blues)
# plt.show()

## Gradiente Descendente Estocástico

In [380]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import learning_curve
sgd = SGDClassifier(loss='perceptron', learning_rate='optimal')
lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr')
scores = cross_val_score(sgd, bag, categorias, cv=10, scoring='accuracy')
print("Acurácia do SGD: %0.2f%%" % (scores.mean()*100))
print("Desvio Padrão: ", scores.std()*2*100)

train_sizes, train_scores, test_scores = learning_curve(estimator=lr, X=X_train, y=y_train, cv=10, train_sizes=np.linspace(0.1, 1.0, 10), n_jobs=1)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.title('Learning Curve')
plt.xlabel('Training Data Size')
plt.ylabel('Model accuracy')
plt.grid()
plt.legend(loc='lower right')
plt.show()

Acurácia do SGD: 93.11%
Desvio Padrão:  2.7844899272194925


KeyboardInterrupt: 